# Live Plotting Cookbook

Real-time visualization of sweep data as it's collected.

Two backends:
- **server**: Plots in browser window (works everywhere)
- **inline**: Plots in notebook cells (this notebook)

## Setup

In [ ]:
import tempfile
import time
import numpy as np
from stanza.logger.data_logger import DataLogger
from stanza.plotter import enable_live_plotting

## Example 1: Basic Inline Plotting

Simplest case - one sweep, inline display.

In [ ]:
# Create logger and enable inline plotting
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="basic", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")

In [ ]:
# Create session and log data
session = logger.create_session()

# First batch - plot appears
x = np.linspace(0, 5, 10)
y = np.sin(x)
session.log_sweep(name="signal", x_data=x, y_data=y, x_label="Time (s)", y_label="Amplitude")
session.flush()
print(f"Logged {len(x)} points")

In [ ]:
# Second batch - same plot updates
x = np.linspace(5, 10, 10)
y = np.sin(x)
session.log_sweep(name="signal", x_data=x, y_data=y, x_label="Time (s)", y_label="Amplitude")
session.flush()
print(f"Total: {len(session._buffer) if hasattr(session, '_buffer') else 'N/A'} points")

In [ ]:
# Cleanup
session.close()

## Example 2: Streaming Data

Simulate real-time measurement with delays.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="streaming", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Stream data in chunks
for i in range(20):
    t = i * 0.5
    amplitude = np.cos(2 * np.pi * 0.5 * t) * np.exp(-t/5)
    
    session.log_sweep(
        name="decay",
        x_data=[t],
        y_data=[amplitude],
        x_label="Time (s)",
        y_label="Signal"
    )
    session.flush()
    time.sleep(0.1)  # Simulate measurement time

print("✓ Streaming complete")

In [ ]:
session.close()

## Example 3: Multiple Plots

Different sweep names create separate plots.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="multi", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Log I and Q components
t = np.linspace(0, 10, 50)

for time_point in t:
    I = np.cos(2 * np.pi * time_point)
    Q = np.sin(2 * np.pi * time_point)
    
    session.log_sweep(name="I", x_data=[time_point], y_data=[I], x_label="Time", y_label="I")
    session.log_sweep(name="Q", x_data=[time_point], y_data=[Q], x_label="Time", y_label="Q")
    session.flush()
    time.sleep(0.05)

print("✓ Two plots created")

In [ ]:
session.close()

## Example 4: Server Backend (Browser)

Use this for longer experiments or when not in a notebook.

In [ ]:
# Setup with server backend
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="server_demo", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="server", port=5006)

print("\n⚠️  Open http://localhost:5006 in your browser NOW\n")

In [ ]:
# Wait a moment for browser to connect
time.sleep(2)

# Now log data - it appears in browser
session = logger.create_session()

for i in range(30):
    x = i * 0.2
    y = np.sin(x) + 0.1 * np.random.randn()
    
    session.log_sweep(
        name="noisy_signal",
        x_data=[x],
        y_data=[y],
        x_label="Time",
        y_label="Signal + Noise"
    )
    session.flush()
    time.sleep(0.1)

print("✓ Check browser for plot")

In [ ]:
session.close()
backend.stop()

## Example 5: Rabi Oscillations

Realistic quantum experiment simulation.

In [ ]:
# Setup
tmpdir = tempfile.mkdtemp()
logger = DataLogger(routine_name="rabi", base_dir=tmpdir)
backend = enable_live_plotting(logger, backend="inline")
session = logger.create_session()

In [ ]:
# Sweep pulse duration
pulse_times = np.linspace(0, 20, 40)  # microseconds

for t_pulse in pulse_times:
    # Simulate Rabi oscillation with decay
    omega_rabi = 2 * np.pi * 0.5  # MHz
    T1 = 30  # microseconds
    
    population = 0.5 * (1 - np.cos(omega_rabi * t_pulse)) * np.exp(-t_pulse / T1)
    
    # Add measurement noise
    population += 0.02 * np.random.randn()
    
    session.log_sweep(
        name="rabi_oscillations",
        x_data=[t_pulse],
        y_data=[population],
        x_label="Pulse Duration (μs)",
        y_label="Excited State Population"
    )
    session.flush()
    time.sleep(0.05)

print("✓ Rabi sweep complete")

In [ ]:
session.close()

## Key Points

1. **Call `flush()`** - Only flushed data appears in plots
2. **Same name = same plot** - Different names create separate plots
3. **Inline backend** - Requires `jupyter_bokeh` package
4. **Server backend** - Open browser to `http://localhost:PORT` before logging
5. **Updates are automatic** - Just flush and watch

## Troubleshooting

**Plot doesn't appear:**
- Did you call `session.flush()`?
- For inline: Is `jupyter_bokeh` installed?
- For server: Is browser open to the right URL?

**Plot doesn't update:**
- Are you using the same sweep name?
- Did you call `flush()` after logging new data?

**Address already in use:**
- Another server on that port
- Use different port: `enable_live_plotting(logger, backend="server", port=5007)`